In [1]:
import pymongo
import re
import pprint
import json
import jieba
from collections import Counter
import time
import math
import logging
from gensim.models import word2vec
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

Using TensorFlow backend.


# 將資料由pymongo讀取近來

In [5]:
#登入pymongo
clinet=pymongo.MongoClient("mongodb://10.120.27.23:27017/")

In [6]:
#確認裡面有哪幾個database
clinet.database_names()

['admin', 'iii-2017-07', 'local', 'mydb', 'rawData', 'test', 'test-database']

In [7]:
#選擇rawData這個db
db=clinet['rawData']

In [8]:
#確認裡面有哪幾個collection
db.collection_names()

['mobile01_tainan',
 'Tainan_final',
 'TainanUnique',
 'merge_tainanUni',
 'ptt_tainan_address',
 'AipinwanFood',
 'ptt_tainan_tel',
 'Tainan_final_name_id_1',
 'pixnet_tainan_author',
 'ptt_tainan']

In [9]:
#將愛評網讀入
ipeenjson=list(db.Tainan_final.find({}))

In [7]:
ipeenjson[0]

{'_id': ObjectId('59afa8e6f3a322258443a5e3'),
 'address': '台南市安南區安北路433號',
 'average_consumption': '42',
 'business_hours': '09:00~23:00',
 'comment': [{'click': 1004,
   'content': '簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji

# 將結巴字典加入自己的許多詞

# 將許多可能會用來分析的項目做成list

In [8]:

#結巴 匯入自己的詞典
jieba.set_dictionary('D:/WordLibrary/JiebaUse/jieba_dict.txt.big')

#停止詞字典stopwordset
stopwordset = set()
with open('D:/WordLibrary/JiebaUse/stopwords.txt', 'r', encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))
        
#店家名稱字典dienlist
dienlist = [dien["name"] for dien in ipeenjson]

#店家種類字典stylelist
stylelist = [dien["style"] for dien in ipeenjson]

#店家種類字典introductionlist
introductionlist = [dien["introduction"] for dien in ipeenjson]

#店家種類字典coordinatelist
coordinatelist = [dien['coordinate'] for dien in ipeenjson]

#自製餐廳相關字字典pinlist
with open("D:\Data\csvtest\MyDict.csv") as f:
    mydict=f.read()
#linewords pindict每個字的評分dict
linewords=[{myword:float(line.split(",")[0]) for myword in line.split(",")[1:]}\
           for line in mydict.split("\n") if line.split(",")[0]!=""]
pindict={}
for word in linewords:
    pindict.update(word)
pinlist=list(pindict.keys())

#將以上字典都加入斷詞行列
for i in dienlist:
    jieba.add_word(i)
for i in stylelist:
    jieba.add_word(i)
for i in pinlist:
    if i !="":
        jieba.add_word(i)
        
#店家評論總數Ncommentlist
Ncommentlist = [len(i["comment"]) for i in ipeenjson]


    

Building prefix dict from D:\WordLibrary\JiebaUse\jieba_dict.txt.big ...
Dumping model to file cache C:\Users\Java\AppData\Local\Temp\jieba.uf3aabe1eaf7c86dda8313189b0c5c683.cache
Loading model cost 1.488 seconds.
Prefix dict has been built succesfully.


In [9]:
#各店家評論diencommentlist
diencommentlist=[[j["content"] for j in i["comment"]] for i in ipeenjson]
diencommentlist[0]

['簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/',
 '來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜',
 '這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是

In [10]:
#各店家回覆dienmessagelist
dienmessagelist=[[''.join(j["message"]) for j in i["comment"] if j["message"]!=None and j["message"]!='null'] for i in ipeenjson]
dienmessagelist[0]

['感謝分享，也幫我推一個，謝謝http://www.ipeen.com.tw/comment/943950',
 '到台南好像不能錯過安平的豆花耶給你非常實用也希望你能給我實用喔【電影類別 / 動作】玩命關頭7-復仇之路 回歸到起點推~~~同是台南人XD這家真的好吃http://www.ipeen.com.tw/comment/838440麻煩支持 一下喔:)',
 '鍋燒意麵這款中山店沒有賣!!!!',
 '竹炭豆畫...黑色的嘛==?感覺鮮奶的比較好吃去安平，必吃美食~~讚！',
 '好像我去過這家！很面熟，但是不太確定。',
 '回推囉，豆花看起來不錯呢！',
 '感覺你們玩超多天的耶XD呵呵 我跟阿左去三天兩夜都感覺沒有你們玩得多呢我喜歡粉圓豆花。這一家商妮去台南玩的時候也有去吃，人好多~',
 '豆花看起來好好吃><紅豆在上面顏色就看起來很漂亮最近的美食分享，喜歡的話幫忙給個讚吧非常謝謝您GODIVA比利時巧克力●奢華享受http://www.ipeen.com.tw/comment/611570IOU Cafe●甜點美饌 http://www.ipeen.com.tw/comment/611714安平豆花我也超愛~下次再來台南~安平豆花地鍋燒意麵和對面的福記水煎包也都蠻好吃的~~很特別的豆花.看起來很嫩那天下去看到好多人，就沒有去排了!!!',
 '每次去安平 我都必吃那家豆花耶~~~超愛!!',
 '我喜歡粉圓加檸檬，酸酸甜甜又清涼上次去台南很猶豫要吃這間還是黑豆花，結果選了黑豆花太太太銷魂了謝謝分享',
 '超愛珍珠豆花的啦~~~~~~檸檬也蠻好吃的~隔壁鍋燒意麵也好吃',
 '好久沒吃了! 上次很像點珍珠檸檬豆花~好吃也喜歡酸甜感我上次也是點檸檬的耶!  紅豆看起來挺不賴的這不是排隊排慘的那家嗎XD豆花好綿啊~看起來就很好吃可惜我上回去~找不到這家，最後隨便路邊找了一間進去吃，結果難吃到爆~~~我也超愛這家豆花，只要有到安平我都會來這家，記的以前他們還沒出名時，我就常去那吃，當時是家小小的店，店裡桌以才3~4張而已呢檸檬的超好吃!!!他們家的鍋燒意面也很不錯~下次可以試試~!\t※ 引述 ✿花露露✿ 之言：\t檸檬的超好吃!!!\t他們家的鍋燒意面也很不錯~下次可以試試~!蝦密??還有賣鍋燒意麵阿?真的假的去很多次了耶~都沒發現....XD\t※

In [11]:
#回覆數量
Nmessagelist=[len(dienmessage) for dienmessage in dienmessagelist]

In [12]:
#全部回覆+全部評論
dienallpinlist=[diencomment+dienmessage for diencomment,dienmessage in zip(diencommentlist,dienmessagelist)]
dienallpinlist[0]


['簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/',
 '來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜',
 '這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是

In [13]:
# for megs in dienmessagelist:
#     for meg in megs:
#         if meg!=None and meg!='null'and meg!='':
#             print(" ".join(meg))

#把所有回覆合併
messagecombinlist=[" ".join(["".join(meg) for meg in megs if meg!=None and meg!='null'and meg!='']) for megs in dienmessagelist]
messagecombinlist[0]

'感謝分享，也幫我推一個，謝謝http://www.ipeen.com.tw/comment/943950 到台南好像不能錯過安平的豆花耶給你非常實用也希望你能給我實用喔【電影類別 / 動作】玩命關頭7-復仇之路 回歸到起點推~~~同是台南人XD這家真的好吃http://www.ipeen.com.tw/comment/838440麻煩支持 一下喔:) 鍋燒意麵這款中山店沒有賣!!!! 竹炭豆畫...黑色的嘛==?感覺鮮奶的比較好吃去安平，必吃美食~~讚！ 好像我去過這家！很面熟，但是不太確定。 回推囉，豆花看起來不錯呢！ 感覺你們玩超多天的耶XD呵呵 我跟阿左去三天兩夜都感覺沒有你們玩得多呢我喜歡粉圓豆花。這一家商妮去台南玩的時候也有去吃，人好多~ 豆花看起來好好吃><紅豆在上面顏色就看起來很漂亮最近的美食分享，喜歡的話幫忙給個讚吧非常謝謝您GODIVA比利時巧克力●奢華享受http://www.ipeen.com.tw/comment/611570IOU Cafe●甜點美饌 http://www.ipeen.com.tw/comment/611714安平豆花我也超愛~下次再來台南~安平豆花地鍋燒意麵和對面的福記水煎包也都蠻好吃的~~很特別的豆花.看起來很嫩那天下去看到好多人，就沒有去排了!!! 每次去安平 我都必吃那家豆花耶~~~超愛!! 我喜歡粉圓加檸檬，酸酸甜甜又清涼上次去台南很猶豫要吃這間還是黑豆花，結果選了黑豆花太太太銷魂了謝謝分享 超愛珍珠豆花的啦~~~~~~檸檬也蠻好吃的~隔壁鍋燒意麵也好吃 好久沒吃了! 上次很像點珍珠檸檬豆花~好吃也喜歡酸甜感我上次也是點檸檬的耶!  紅豆看起來挺不賴的這不是排隊排慘的那家嗎XD豆花好綿啊~看起來就很好吃可惜我上回去~找不到這家，最後隨便路邊找了一間進去吃，結果難吃到爆~~~我也超愛這家豆花，只要有到安平我都會來這家，記的以前他們還沒出名時，我就常去那吃，當時是家小小的店，店裡桌以才3~4張而已呢檸檬的超好吃!!!他們家的鍋燒意面也很不錯~下次可以試試~!\t※ 引述 ✿花露露✿ 之言：\t檸檬的超好吃!!!\t他們家的鍋燒意面也很不錯~下次可以試試~!蝦密??還有賣鍋燒意麵阿?真的假的去很多次了耶~都沒發現....XD\t※ 引述 品味廣東粥 之言：\t我也超愛這家豆花，只要有到安平我都會來這家，記的以前他們還沒

In [14]:
#把所有評論合併
commentcombinelist = [" ".join([j["content"] for j in i["comment"]]) for i in ipeenjson]
commentcombinelist[0]

'簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/ 來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜 這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是平日的暑假店裡還是

In [15]:
#合併評論+回覆
pincombinelist=[comment+" "+message for comment,message in zip(commentcombinelist,messagecombinlist)]
pincombinelist[0]

'簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/ 來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜 這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是平日的暑假店裡還是

# 做TD-IDF分析

In [16]:

st=time.time()
#取得各店家的斷完詞評論pincutlist
pincutlist = []
cutcount=0
for pincombine in pincombinelist:
    cutcount+=1
    if cutcount%200==0:
        print("以切"+str(cutcount)+"篇文章的詞")
    words = jieba.cut(pincombine, cut_all=False)
    pincut=" ".join([word for word in words if word not in stopwordset and '\u4e00' <= word and word <= '\u9fff'])
    pincutlist.append(pincut)
    
#所有店家的用詞加總前三百wordCountlist300
wordCountlist=[Counter(pincut.split(" ")).most_common(300) for pincut in pincutlist]

#TF計算 (TF=這個字出現的次數/這篇文章的總字數)
#(sum(dict(wordCount).values())=>一篇所有字的字數  wordkv[1]>>該文字字數  wordkv[0]>>該文字)
#TFList每篇文章的詞 的詞頻
TFList=[[((wordkv[0],wordkv[1]/sum(dict(wordCount).values()))) for wordkv in wordCount] for wordCount in wordCountlist]


#全文章共用了幾個詞(set可以去除重複，可迭代)
wordlist=[]
for wordCount in wordCountlist:
    for word in wordCount:
        wordlist.append(word[0])
wordlist=set(wordlist)

#nDien所有的文章(店家)數
ndien=len(dienlist)

#wordappear計算每個字出現於 (幾篇) 文章
wordappear={}
appearcount=0
for word in wordlist:
    appearcount+=1
    if appearcount%1000==0:
        print("以計算"+str(appearcount)+"個文字的出現篇數")
    n=0
    for pincut in pincutlist:
        if word in pincut:
            n+=1
    wordappear[word]=n
    
#IDF(逆向檔案頻率)=某詞  所有文章數/在幾篇文章出現 開log10 >>次方數為所
IDFlist={word:math.log(ndien/wordappear[word],10) for word in wordappear}

#dienTF每家店的全字TF值 wordTF每個詞的TF   wordTF[0]詞 wordTF[1]詞的TF值  IDFlist[wordTF[0]]詞的IDF值
TF_IDFlist=[Counter({wordTF[0]:wordTF[1]*IDFlist[wordTF[0]]for wordTF in dienTF})for dienTF in TFList]
for name,TFIDF in zip(dienlist,TF_IDFlist):
    if name in TFIDF:
        del TFIDF[name]


et=time.time()
print(et-st)

以切200篇文章的詞
以切400篇文章的詞
以切600篇文章的詞
以切800篇文章的詞
以切1000篇文章的詞
以切1200篇文章的詞
以切1400篇文章的詞
以切1600篇文章的詞
以切1800篇文章的詞
以切2000篇文章的詞
以切2200篇文章的詞
以切2400篇文章的詞
以切2600篇文章的詞
以切2800篇文章的詞
以切3000篇文章的詞
以切3200篇文章的詞
以切3400篇文章的詞
以切3600篇文章的詞
以切3800篇文章的詞
以切4000篇文章的詞
以切4200篇文章的詞
以切4400篇文章的詞
以切4600篇文章的詞
以切4800篇文章的詞
以切5000篇文章的詞
以切5200篇文章的詞
以切5400篇文章的詞
以切5600篇文章的詞
以計算1000個文字的出現篇數
以計算2000個文字的出現篇數
以計算3000個文字的出現篇數
以計算4000個文字的出現篇數
以計算5000個文字的出現篇數
以計算6000個文字的出現篇數
以計算7000個文字的出現篇數
以計算8000個文字的出現篇數
以計算9000個文字的出現篇數
以計算10000個文字的出現篇數
以計算11000個文字的出現篇數
以計算12000個文字的出現篇數
以計算13000個文字的出現篇數
以計算14000個文字的出現篇數
以計算15000個文字的出現篇數
以計算16000個文字的出現篇數
以計算17000個文字的出現篇數
以計算18000個文字的出現篇數
以計算19000個文字的出現篇數
以計算20000個文字的出現篇數
以計算21000個文字的出現篇數
以計算22000個文字的出現篇數
以計算23000個文字的出現篇數
以計算24000個文字的出現篇數
以計算25000個文字的出現篇數
以計算26000個文字的出現篇數
以計算27000個文字的出現篇數
以計算28000個文字的出現篇數
以計算29000個文字的出現篇數
以計算30000個文字的出現篇數
以計算31000個文字的出現篇數
以計算32000個文字的出現篇數
以計算33000個文字的出現篇數
以計算34000個文字的出現篇數
以計算35000個文字的出現篇數
以計算36000個文字的出現篇數
以計算37000個文字的出現篇數
以計算38000個文字的出現篇數
以計算39000個文字的出現篇數
以計算40000個文字的出現

# 將上面的斷詞文章combine在一起 做word2vec model

In [18]:


#將所有的斷詞文章combine在一起，變成超大斷詞字串，並存起來
contentcutcombine=" ".join(pincutlist)
with open('D:\Data\word2VecData\TainanContent.txt','w',encoding="utf-8") as w:
    w.write(contentcutcombine)
    
#將存起來的文章用word2vec.Text8Corpus切成有順序的詞彙list(sentences)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = word2vec.Text8Corpus("D:/Data/word2VecData/TainanContent.txt")

#將有順序的詞彙list丟給word2vec做訓練
model = word2vec.Word2Vec(sentences, size=300,window=6, min_count=4, workers=4,sg=1)
# sentences:當然了，這是要訓練的句子集，沒有他就不用跑了
# size:這表示的是訓練出的詞向量會有幾維
# alpha:機器學習中的學習率，這東西會逐漸收斂到 min_alpha
# sg:sg=1表示採用skip-gram(由本字去測他字 擴散),sg=0表示採用cbow(由他字測本字 集中)
# window:能往左往右看幾個字的意思
# workers:執行緒數目，除非電腦不錯，不然建議別超過 4
# min_count:若這個詞出現的次數小於min_count，那他就不會被視為訓練對象

# Save our model.
model.save("D:/Data/word2VecData/Tainan.model.bin")


2017-09-12 01:30:43,936 : INFO : collecting all words and their counts
2017-09-12 01:30:43,939 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-09-12 01:30:48,754 : INFO : collected 221611 word types from a corpus of 12723753 raw words and 1273 sentences
2017-09-12 01:30:48,755 : INFO : Loading a fresh vocabulary
2017-09-12 01:30:49,191 : INFO : min_count=4 retains 83521 unique words (37% of original 221611, drops 138090)
2017-09-12 01:30:49,192 : INFO : min_count=4 leaves 12521847 word corpus (98% of original 12723753, drops 201906)
2017-09-12 01:30:49,483 : INFO : deleting the raw counts dictionary of 221611 items
2017-09-12 01:30:49,494 : INFO : sample=0.001 downsamples 20 most-common words
2017-09-12 01:30:49,495 : INFO : downsampling leaves estimated 12172806 word corpus (97.2% of prior 12521847)
2017-09-12 01:30:49,496 : INFO : estimated required memory for 83521 words and 300 dimensions: 242210900 bytes
2017-09-12 01:30:49,827 : INFO : resetting la

2017-09-12 01:32:04,335 : INFO : PROGRESS: at 24.48% examples, 203352 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:32:05,372 : INFO : PROGRESS: at 24.85% examples, 203604 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:32:06,375 : INFO : PROGRESS: at 25.22% examples, 203814 words/s, in_qsize 7, out_qsize 0
2017-09-12 01:32:07,470 : INFO : PROGRESS: at 25.56% examples, 203650 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:32:08,477 : INFO : PROGRESS: at 25.92% examples, 203843 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:32:09,499 : INFO : PROGRESS: at 26.27% examples, 203868 words/s, in_qsize 7, out_qsize 0
2017-09-12 01:32:10,557 : INFO : PROGRESS: at 26.63% examples, 203923 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:32:11,591 : INFO : PROGRESS: at 26.99% examples, 204036 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:32:12,622 : INFO : PROGRESS: at 27.35% examples, 204154 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:32:13,682 : INFO : PROGRESS: at 27.71% examples, 204195 wor

2017-09-12 01:33:27,509 : INFO : PROGRESS: at 52.57% examples, 204546 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:33:28,523 : INFO : PROGRESS: at 52.98% examples, 204804 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:33:29,524 : INFO : PROGRESS: at 53.37% examples, 205025 words/s, in_qsize 7, out_qsize 0
2017-09-12 01:33:30,591 : INFO : PROGRESS: at 53.76% examples, 205152 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:33:31,655 : INFO : PROGRESS: at 54.17% examples, 205340 words/s, in_qsize 7, out_qsize 1
2017-09-12 01:33:32,710 : INFO : PROGRESS: at 54.58% examples, 205537 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:33:33,715 : INFO : PROGRESS: at 54.97% examples, 205734 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:33:34,715 : INFO : PROGRESS: at 55.33% examples, 205824 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:33:35,725 : INFO : PROGRESS: at 55.73% examples, 206013 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:33:36,775 : INFO : PROGRESS: at 56.15% examples, 206267 wor

2017-09-12 01:34:50,452 : INFO : PROGRESS: at 82.91% examples, 210793 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:34:51,514 : INFO : PROGRESS: at 83.30% examples, 210858 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:34:52,536 : INFO : PROGRESS: at 83.68% examples, 210917 words/s, in_qsize 6, out_qsize 1
2017-09-12 01:34:53,537 : INFO : PROGRESS: at 84.08% examples, 211075 words/s, in_qsize 7, out_qsize 0
2017-09-12 01:34:54,575 : INFO : PROGRESS: at 84.48% examples, 211158 words/s, in_qsize 6, out_qsize 1
2017-09-12 01:34:55,576 : INFO : PROGRESS: at 84.87% examples, 211273 words/s, in_qsize 6, out_qsize 1
2017-09-12 01:34:56,588 : INFO : PROGRESS: at 85.28% examples, 211416 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:34:57,638 : INFO : PROGRESS: at 85.66% examples, 211447 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:34:58,673 : INFO : PROGRESS: at 86.06% examples, 211567 words/s, in_qsize 8, out_qsize 0
2017-09-12 01:34:59,679 : INFO : PROGRESS: at 86.41% examples, 211556 wor

# 試玩word2vec的model  需解除#

In [19]:
# from gensim.models import word2vec
# from gensim import models
# import logging

# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# model = models.Word2Vec.load('D:/Data/word2VecData/Tainan.model.bin')

# print("提供 3 種測試模式\n")
# print("輸入一個詞，則去尋找前一百個該詞的相似詞")
# print("輸入兩個詞，則去計算兩個詞的餘弦相似度")
# print("輸入三個詞，進行類比推理")

# while True:
#     try:
#         query = input()
#         q_list = query.split()

#         if len(q_list) == 1:
#             print("相似詞前 30 排序")
#             res = model.most_similar(q_list[0],topn = 30)
#             for item in res:
#                 print(item[0]+","+str(item[1]))

#         elif len(q_list) == 2:
#             print("計算 Cosine 相似度")
#             res = model.similarity(q_list[0],q_list[1])
#             print(res)
#         else:
#             print("%s之於%s，如%s之於" % (q_list[0],q_list[2],q_list[1]))
#             res = model.most_similar([q_list[0],q_list[1]], [q_list[2]], topn= 20)
#             for item in res:
#                 print(item[0]+","+str(item[1]))
#         print("----------------------------")
#     except Exception as e:
#         print(repr(e))


In [2]:
pincutlist[0]

NameError: name 'pincutlist' is not defined

# 試Rocchio、bayes、kmeans、SVM分群 (仿白話大數據寫法)

In [20]:
#取得各店家的斷完詞評論contentcutlist
#店家種類字典stylelist
# from sklearn.datasets import fetch_20newsgroups
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer

#斷詞function
def cut_text_list(textlist):
    textcutlist=[]
    for text in textlist:
        cutwords=jieba.cut(text)
        cuttext=" ".join([word for word in cutwords if word not in stopwordset and '\u4e00' <= word <= '\u9fff'])
        textcutlist.append(cuttext)
    return textcutlist

#文字轉向量矩陣
count_vect=CountVectorizer()
X_train_counts=count_vect.fit_transform(pincutlist)
print('文字轉向量矩陣')

#向量矩陣做tfidf

tfidf_transformer=TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print('tfidf')



#ptt測試集|
with open(r'D:\Data\testData\foodptt.json') as f:
    testdatas=json.load(f)
docs_new=cut_text_list(testdatas)
X_new_counts= count_vect.transform(docs_new)
x_new_tfidf=tfidf_transformer.transform(X_new_counts)

#試用Roccio分群
# from sklearn.neighbors.nearest_centroid import NearestCentroid
st=time.time()
clf = NearestCentroid().fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("Roccio")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)


#試用bayes分群
# from sklearn.naive_bayes import MultinomialNB
st=time.time()
clf = MultinomialNB().fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("bayes")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)

#試用kmeans分群
# from sklearn.neighbors import KNeighborsClassifier
st=time.time()
clf = KNeighborsClassifier(15).fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("Kmeans")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)

#試用svm分群
# from sklearn import svm
st=time.time()
clf = svm.SVC(kernel="linear").fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("svm")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)

文字轉向量矩陣
tfidf
Roccio
array(['台菜餐廳', '美式料理', '義式料理', '西點烘焙、麵包', '其他鍋類', '', '臭豆腐', '', '複合式',
       '美式料理', '其他小吃', '泰式料理', '台菜餐廳'],
      dtype='<U13')
0.8686103820800781
bayes
array(['其他小吃', '複合式', '複合式', '複合式', '複合式', '其他小吃', '其他小吃', '其他小吃', '複合式',
       '其他小吃', '其他小吃', '其他小吃', '其他小吃'],
      dtype='<U13')
2.0297577381134033
Kmeans
array(['複合式', '咖啡專賣', '其他小吃', '飲料專賣店', '其他小吃', '其他日式料理', '其他小吃', '其他小吃',
       '咖啡專賣', '咖啡專賣', '炭烤串燒', '其他小吃', '西點烘焙、麵包'],
      dtype='<U13')
0.5380682945251465
svm
array(['其他小吃', '複合式', '義式料理', '西點烘焙、麵包', '其他小吃', '其他小吃', '臭豆腐', '其他小吃',
       '複合式', '其他小吃', '其他小吃', '泰式料理', '其他小吃'],
      dtype='<U13')
272.3290812969208


# 自己肉眼辨識的答案是 合菜 美式 義式 西點 鍋類 冰品 臭豆腐 其他 義式 美式 小吃 泰式 台菜

# 最準的是 Roccio > svm > kmeans > bayes

In [21]:
#參考PTT斷詞文章
docs_new

['壹等賞 桃園 龍潭 看板 批 踢踢 實業 坊 批 踢踢 實業 坊 看板 聯絡 資訊 返回 看板 分享 作者 看板 標題 壹等賞 桃園 龍潭 時間 本文 轉錄 看板 作者 看板 標題 壹等賞 桃園 龍潭 時間 圖文 版 今天 提早到 新竹 火車站 附近 髮 瑟 剪 頭髮 整個 人 變得 清爽 免不了 念 幾句 想 以後 頭髮 剪 好 今天 比較 晚 下班 六點 多才 離開 公司 交通 正 處於 尖峰 時刻 光復路 繞 一圈 還了 漫畫 之後 決定 晚上 吃 桃園 龍潭 壹等賞 新竹 文化 街上 紅葉 告訴 目前 肚子 不餓 想 說 現在 開 壹等賞 時間 應該 夠長 肚子 應該 餓 決定 上次 曾經 吃過 一次 壹等賞 享用 晚餐 壹等賞 位於 龍潭 渴望 園區 附近 上次 來過 一次 不知 道路 走 知道 高速公路 累得 睡著 開 十分鐘 下 關西 交 流道 辦法 睡 一下 一下 交流 道 吵醒 請 幫 帶路 今 天 再 走過 一次 不太 認得 路 晚上 天色 黑 另一方 面是 山路 白天 去過 難記 住 路 走 下面 名片 背後 地圖 有興 趣 參考 一下 地址 桃園縣 龍潭 鄉 高原 村 中原 路 弄 電話 營業時間 每週 公休 今天 一到 停車場 滿滿的 車子 已經 七點 多了還 一堆 人 一看 發現 應該 公司 聚餐 很多人 穿著 同樣 制服 外面 抽 飯後 煙 空間 很大 晚到 位置 坐 壹等賞 一個 茶園 旁邊 一片 很大 茶 田 裡面 餐廳 賣 茶葉 東西 就連 洗手間 洗手 乳 茶葉 香味 服務 人員 一間 廂房 裡面 已經 坐著 滿滿的 客人 一到 看到 客人 本來 已經 心理準備 廚房 上菜 比較慢 一點 一點 完菜 不到 三分鐘 菜 一道 一道 送上來 真 佩服 廚房 裡面 大廚 燒菜 功力 上菜 食物 好吃 熱茶 免費 餐桌上 一壺 大壺 熱茶 免費 提供 飲用 醃菜 頭 免費 很喜歡 吃 只 吃 兩塊 剩下 吃 光光 用來 搭配 稀飯 一起 吃 最好 稀飯 點 好吃 茶油 拌 飯 珍香蜜 茶 鵝 小 茶鵝 很好吃 有點 貴 外面 賣 鵝肉 不太 應該 有用 茶葉 燻 很香 養生茶 油拌 飯 點了 兩碗 四碗 普通 碗 份量 要點 茶油 拌 飯 吃 哪有 吃白飯 道理 桂花 蜜 排骨 小 糖醋排骨 吃 覺得 還不錯 吃 最後 有點 吃 不

In [43]:
dienallpinlist[0]

['簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/',
 '來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜',
 '這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是

# 試者用自己的字典給店家分數(一篇評論的正負評價詞最多算5個)

In [22]:
#各店家評論diencommentlist
#取得各店家的斷完詞評論contentcutlist
#自製餐廳相關字字典pinlist
#pindict每個字的評分dict
#各店家評論diencommentlist

#pinCountMaxper5list 一個店家出現的餐廳相關詞數量(一個篇評論的相同詞最多給5個分數)
st=time.time()
pinCountMaxper5list = []
countN=0
for dienallpin in dienallpinlist:
    dienWordcount={}
    for allpin in  dienallpin:
        countN+=1
        if countN%100==0:
            print("已算完"+str(countN)+"篇情緒字")
        for pinword in pinlist:
            if len(re.findall(pinword,allpin))>0:
                if len(re.findall(pinword,allpin))>5:
                    npinword=5
                else:
                    npinword=len(re.findall(pinword,allpin))
                if pinword not in dienWordcount:
                    dienWordcount[pinword]=0
                dienWordcount[pinword]+=npinword
    pinCountMaxper5list.append(dienWordcount)
ed=time.time()
print(ed-st)

已算完100篇情緒字
已算完200篇情緒字
已算完300篇情緒字
已算完400篇情緒字
已算完500篇情緒字
已算完600篇情緒字
已算完700篇情緒字
已算完800篇情緒字
已算完900篇情緒字
已算完1000篇情緒字
已算完1100篇情緒字
已算完1200篇情緒字
已算完1300篇情緒字
已算完1400篇情緒字
已算完1500篇情緒字
已算完1600篇情緒字
已算完1700篇情緒字
已算完1800篇情緒字
已算完1900篇情緒字
已算完2000篇情緒字
已算完2100篇情緒字
已算完2200篇情緒字
已算完2300篇情緒字
已算完2400篇情緒字
已算完2500篇情緒字
已算完2600篇情緒字
已算完2700篇情緒字
已算完2800篇情緒字
已算完2900篇情緒字
已算完3000篇情緒字
已算完3100篇情緒字
已算完3200篇情緒字
已算完3300篇情緒字
已算完3400篇情緒字
已算完3500篇情緒字
已算完3600篇情緒字
已算完3700篇情緒字
已算完3800篇情緒字
已算完3900篇情緒字
已算完4000篇情緒字
已算完4100篇情緒字
已算完4200篇情緒字
已算完4300篇情緒字
已算完4400篇情緒字
已算完4500篇情緒字
已算完4600篇情緒字
已算完4700篇情緒字
已算完4800篇情緒字
已算完4900篇情緒字
已算完5000篇情緒字
已算完5100篇情緒字
已算完5200篇情緒字
已算完5300篇情緒字
已算完5400篇情緒字
已算完5500篇情緒字
已算完5600篇情緒字
已算完5700篇情緒字
已算完5800篇情緒字
已算完5900篇情緒字
已算完6000篇情緒字
已算完6100篇情緒字
已算完6200篇情緒字
已算完6300篇情緒字
已算完6400篇情緒字
已算完6500篇情緒字
已算完6600篇情緒字
已算完6700篇情緒字
已算完6800篇情緒字
已算完6900篇情緒字
已算完7000篇情緒字
已算完7100篇情緒字
已算完7200篇情緒字
已算完7300篇情緒字
已算完7400篇情緒字
已算完7500篇情緒字
已算完7600篇情緒字
已算完7700篇情緒字
已算完7800篇情緒字
已算完7900篇情緒字
已算完8000篇情緒字
已算完8100篇情緒字
已算完8200篇情緒字
已算完8300篇情緒字
已算完8400篇情緒字
已

In [46]:
pinCountMaxper5list[50]

{'': 432,
 'Q彈': 1,
 '一品': 3,
 '一般般': 1,
 '不太喜歡': 2,
 '不好吃': 2,
 '不愛': 4,
 '不膩口': 4,
 '也不錯': 8,
 '乾了點': 1,
 '乾淨': 5,
 '亂': 5,
 '享受': 9,
 '偏酸': 1,
 '健康': 9,
 '入口即化': 1,
 '划算': 5,
 '剛好': 16,
 '口感紮實': 1,
 '古色古香': 1,
 '可愛': 20,
 '吃飽': 1,
 '合理': 2,
 '喜歡': 45,
 '大份': 2,
 '大碗': 3,
 '天然': 8,
 '天然食材': 2,
 '天龍': 4,
 '太多了': 3,
 '太甜': 14,
 '太臭': 1,
 '好吃': 96,
 '好喝': 4,
 '好好吃': 12,
 '好有味道': 2,
 '好貴': 3,
 '好食': 1,
 '實惠': 1,
 '寬敞': 1,
 '小吃': 24,
 '小巧': 4,
 '小貴': 1,
 '平價': 3,
 '平心而論': 1,
 '很不錯': 3,
 '很便宜': 3,
 '很喜歡': 7,
 '很好吃': 16,
 '很有禮貌': 1,
 '很棒': 1,
 '很清爽': 2,
 '很濃郁': 10,
 '很甜': 1,
 '很美味': 1,
 '很脆': 1,
 '很膩': 1,
 '很難吃': 1,
 '很香': 4,
 '後悔': 2,
 '態度好': 1,
 '整齊': 4,
 '新鮮': 11,
 '明亮': 5,
 '有禮貌': 1,
 '有落差': 1,
 '有點小': 2,
 '有點硬': 3,
 '柔軟': 3,
 '浪漫': 2,
 '清幽': 1,
 '清爽': 16,
 '溫暖': 3,
 '滿喜歡': 1,
 '滿意': 1,
 '漂亮': 4,
 '漲成': 1,
 '濃郁': 42,
 '熱呼呼的': 1,
 '熱情': 6,
 '爽口': 1,
 '特別': 21,
 '甜度恰到好處': 2,
 '精緻': 6,
 '精美': 3,
 '經濟實惠': 1,
 '綿密': 21,
 '綿密口感': 1,
 '綿綿的': 2,
 '缺陷': 1,
 '美味': 16,
 '脆脆的': 3,
 '膩口': 4,
 '裝潢': 

In [24]:
#店家評論總數Ncommentlist
#店家回覆總數Nmessage
#將六篇回覆視為一篇文章，做正規化(防止評論數過多的店家分數極端化)
Npartpin=[int(Nmessage/6)+Ncomment for Nmessage,Ncomment in zip(Nmessagelist,Ncommentlist)]


#大致計算每家店的分數dienscorelist
dienscorelist=[]
for pinCount,N in zip(pinCountMaxper5list,Npartpin):
    dienscore=0
    for pin in pinCount:
        dienscore+=pinCount[pin]*pindict[pin]/N
    dienscorelist.append(dienscore)

In [25]:
dienscorelist[2]

8.130337078651682

In [26]:
#mydict為自己的字典的資料
#格式為:
######否開頭tag 為 非否tag 反義
#給予正分數 tag評價詞(非否評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予負分數 tag評價詞(否開頭評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予正分數 tag評價詞(非否評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予負分數 tag評價詞(否開頭評價) 同意字群同意字群同意字群同意字群同意字群同意字群

#line.split(",")[1]為tag!!! word為與tag對應的同義字
synonyms=[{word:line.split(",")[1] for word in line.split(",")[1:]} for line in mydict.split("\n") if line.split(",")[0]!=""]

pinsynonymsdict={}
for synonym in synonyms:
    pinsynonymsdict.update(synonym)
del pinsynonymsdict[""]




In [27]:
pinsynonymsdict

{'CP值很低': '否划算',
 'CP值非常高': '划算',
 'CP值高份量夠': '划算',
 'Q彈': '食物美味',
 'Q彈柔嫩': '食物美味',
 'cp值不高': '否划算',
 'cp值太低': '否划算',
 'cp值很高': '划算',
 'hen好吃': '食物美味',
 'juicy': '食物美味',
 '一個人吃剛剛好': '小份量',
 '一口想把它喝光的衝動': '食物美味',
 '一品': '食物美味',
 '一問三不知': '否服務好',
 '一塵不染': '環境好',
 '一定要稱讚一下': '食物美味',
 '一樣無敵': '食物美味',
 '一致好評': '食物美味',
 '一般般': '否食物美味',
 '不乾不柴': '食物美味',
 '不乾柴': '食物美味',
 '不乾淨': '否環境好',
 '不划算': '否划算',
 '不卡牙': '食物美味',
 '不可多得': '服務好',
 '不夠酥脆': '否食物美味',
 '不太優': '否食物美味',
 '不太喜歡': '否食物美味',
 '不太好吃': '否食物美味',
 '不太愛吃': '否食物美味',
 '不太慣': '否食物美味',
 '不好吃': '否食物美味',
 '不對味': '否食物美味',
 '不怎麼樣': '否食物美味',
 '不想離開': '其他正面情緒',
 '不愛': '否食物美味',
 '不愛吃': '否食物美味',
 '不愛它': '否食物美味',
 '不搭': '否食物美味',
 '不敢恭維': '否環境好',
 '不斐': '否划算',
 '不新鮮': '否食物美味',
 '不會再來': '否其他正面情緒',
 '不會去第二次': '否其他正面情緒',
 '不會吃不起': '划算',
 '不會太重': '口味清淡',
 '不會太鹹': '口味清淡',
 '不會過於甜膩': '食物美味',
 '不會過甜': '食物美味',
 '不會過膩': '食物美味',
 '不油膩': '食物美味',
 '不熱': '否食物美味',
 '不用擔心': '其他正面情緒',
 '不用贅述就是好吃阿': '食物美味',
 '不算便宜': '否划算',
 '不脆': '否食物美味',
 '不膩口': '食物美味',
 '不親民': '否划算',


In [28]:
#每篇文章pinCountMaxper5list評價詞出現次數
#評價同義詞pinsynonymsdict

#取得各餐廳的評價特徵分數featurescores!!!
featurescores=[]

#將評價詞加總及評論次數做iterate
for pinCountMaxper5,N in zip(pinCountMaxper5list,Npartpin):
    dienfeaturescore={}
    
    # 評價詞出現次數/評論次數 做為分數，其中否字開頭給負分
    for pin in pinCountMaxper5:
        if pin !="":
            if pinsynonymsdict[pin][0]=="否":
                if pinsynonymsdict[pin][1:] not in dienfeaturescore:
                    dienfeaturescore[pinsynonymsdict[pin][1:]]=0
                dienfeaturescore[pinsynonymsdict[pin][1:]]-=round(pinCountMaxper5[pin]/N,2)*1
            else:
                if pinsynonymsdict[pin] not in dienfeaturescore:
                    dienfeaturescore[pinsynonymsdict[pin]]=0
                dienfeaturescore[pinsynonymsdict[pin]]+=round(pinCountMaxper5[pin]/N,2)
    if len(dienfeaturescore)==0:
        dienfeaturescore["無評"]=1
    featurescores.append(dienfeaturescore)
    
    
    

# 將上述屬性結合成一個大分析表

In [29]:
BigAnalyzeTable=[]
for name,style,Ncomment,contentcut,wordCount,TFIDF,pinCountMax5,score,featurescore,introduction,coordinate in zip(dienlist,stylelist,Npartpin,pincutlist,wordCountlist,TF_IDFlist,pinCountMaxper5list,dienscorelist,featurescores,introductionlist,coordinatelist):
    dien={}
    dien['name']=name
    dien['style']=style
    dien['Ncomment']=Ncomment
    dien['contentcut']=contentcut
    dien['wordCount']=wordCount
    dien['TF_IDF']=TFIDF
    dien['pinCountMaxper5']=pinCountMax5
    dien['score']=score
    dien['featurescores']=featurescore
    dien['tags']=[tag[0] for tag in Counter(TFIDF).most_common(5)]
    
    dien['introduction']=introduction
    dien['coordinate']=coordinate
    
    
    BigAnalyzeTable.append(dien)


# dienlist
# stylelist
# Ncommentlist
# pincutlist
# wordCountlist
# TF_IDFlist
# pinCountMaxper5list
# dienscorelist
# featurescores


#店家種類字典introductionlist
# introductionlist = [dien["introduction"] for dien in ipeenjson]

#店家種類字典coordinatelist
# coordinatelist = [dien['coordinate'] for dien in ipeenjson]

# 將每個自定義的評價詞做正規化評分、並且將分數達標者做為標籤貼上!

In [30]:
#算出每個評論的評論數N時的 最大值b 最小值g for正規化


#allpin=評論詞list
allpin=set([line.split(",")[1] for line in mydict.split("\n") if line.split(",")[0]!="" and line.split(",")[1][0]!="否"])
standard={}

#standard為正規化所需用的值
for pin in allpin:
    for rang in [(0,5),(5,10),(10,15),(15,100)]:
        get={}
        values=[dien['featurescores'][pin] for dien in BigAnalyzeTable if pin in dien['featurescores'] and dien["Ncomment"]>rang[0] and dien["Ncomment"]<=rang[1]]
        b=0
        g=0
        for value in values:
            if value <b:
                b=round(value,1)
            if value >g:
                g=round(value,1)
        get[pin+str(rang[0])]=((g+b)/2,g-(g+b)/2)
        standard.update(get)
        print(pin+"+"+str(rang[0])+"\t"+str(g)+"\t"+str(b))

偏甜+0	5.0	0
偏甜+5	1.3	0
偏甜+10	1.1	0
偏甜+15	1.6	0
份量大+0	9.0	0
份量大+5	2.7	0
份量大+10	3.5	0
份量大+15	3.1	0
環境好+0	26.0	-3.0
環境好+5	6.7	-0.6
環境好+10	6.1	-0.5
環境好+15	5.3	-0.3
偏辣+0	5.0	0
偏辣+5	1.3	0
偏辣+10	0.9	0
偏辣+15	1.3	0
食物精緻漂亮+0	15.0	0
食物精緻漂亮+5	7.2	0
食物精緻漂亮+10	4.5	0
食物精緻漂亮+15	6.0	0
口味清淡+0	3.0	0
口味清淡+5	0.8	0
口味清淡+10	0.5	0
口味清淡+15	1.1	0
偏酸+0	1.0	0
偏酸+5	0.5	0
偏酸+10	0.3	0
偏酸+15	0.3	0
服務好+0	3.0	-3.0
服務好+5	1.9	-0.5
服務好+10	1.0	-0.2
服務好+15	0.9	-0.2
食物很香+0	8.0	0
食物很香+5	3.7	0
食物很香+10	2.3	0
食物很香+15	4.2	0
其他正面情緒+0	18.0	-2.0
其他正面情緒+5	7.3	-0.3
其他正面情緒+10	9.6	0
其他正面情緒+15	8.0	0
划算+0	10.0	-9.0
划算+5	5.3	-4.9
划算+10	4.1	-1.7
划算+15	2.3	-2.1
小份量+0	7.0	0
小份量+5	3.0	0
小份量+10	2.5	0
小份量+15	3.1	0
食物美味+0	37.0	-19.0
食物美味+5	23.0	0
食物美味+10	19.9	0
食物美味+15	21.6	0
重口味+0	5.0	0
重口味+5	1.7	0
重口味+10	1.3	0
重口味+15	1.0	0


In [31]:
standard

{'份量大0': (4.5, 4.5),
 '份量大10': (1.75, 1.75),
 '份量大15': (1.55, 1.55),
 '份量大5': (1.35, 1.35),
 '偏甜0': (2.5, 2.5),
 '偏甜10': (0.55, 0.55),
 '偏甜15': (0.8, 0.8),
 '偏甜5': (0.65, 0.65),
 '偏辣0': (2.5, 2.5),
 '偏辣10': (0.45, 0.45),
 '偏辣15': (0.65, 0.65),
 '偏辣5': (0.65, 0.65),
 '偏酸0': (0.5, 0.5),
 '偏酸10': (0.15, 0.15),
 '偏酸15': (0.15, 0.15),
 '偏酸5': (0.25, 0.25),
 '其他正面情緒0': (8.0, 10.0),
 '其他正面情緒10': (4.8, 4.8),
 '其他正面情緒15': (4.0, 4.0),
 '其他正面情緒5': (3.5, 3.8),
 '划算0': (0.5, 9.5),
 '划算10': (1.1999999999999997, 2.9),
 '划算15': (0.09999999999999987, 2.2),
 '划算5': (0.19999999999999973, 5.1),
 '口味清淡0': (1.5, 1.5),
 '口味清淡10': (0.25, 0.25),
 '口味清淡15': (0.55, 0.55),
 '口味清淡5': (0.4, 0.4),
 '小份量0': (3.5, 3.5),
 '小份量10': (1.25, 1.25),
 '小份量15': (1.55, 1.55),
 '小份量5': (1.5, 1.5),
 '服務好0': (0.0, 3.0),
 '服務好10': (0.4, 0.6),
 '服務好15': (0.35, 0.55),
 '服務好5': (0.7, 1.2),
 '環境好0': (11.5, 14.5),
 '環境好10': (2.8, 3.3),
 '環境好15': (2.5, 2.8),
 '環境好5': (3.0500000000000003, 3.65),
 '重口味0': (2.5, 2.5),
 '重口味10': (0.65, 0.65

In [32]:
#進行正規化的處理
for dien in BigAnalyzeTable:
    N=dien['Ncomment']
    for feature in dien['featurescores']:
        if feature !="無評":
            if N<6:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(0)][1],2)
            elif N<11 and N>=6:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(5)][1],2)
            elif N<16 and N>=11:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(10)][1],2)
            else:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(15)][1],2)

# 正規化後，每個屬性的評價區間都為2! 但未避免有些沒被負評的店判斷成負評，不將中間值移至0的位置

In [33]:


allpin=[line.split(",")[1] for line in mydict.split("\n") if line.split(",")[0]!="" and line.split(",")[1][0]!="否"]
standard={}

for pin in allpin:
    for rang in [(0,5),(5,10),(10,15),(15,100)]:
#         get={}
        values=[dien['featurescores'][pin] for dien in BigAnalyzeTable if pin in dien['featurescores'] and dien["Ncomment"]>rang[0] and dien["Ncomment"]<=rang[1]]
        b=0
        g=0
        for value in values:
            if value <b:
                b=round(value,1)
            if value >g:
                g=round(value,1)
#         get[pin+str(rang[0])]=((g+b)/2,g-(g+b)/2)
#         standard.update(get)
        print(pin+"+"+str(rang[0])+"\t"+str(g)+"\t"+str(b))

食物美味+0	1.3	-0.7
食物美味+5	2.0	0
食物美味+10	2.0	0
食物美味+15	2.0	0
食物美味+0	1.3	-0.7
食物美味+5	2.0	0
食物美味+10	2.0	0
食物美味+15	2.0	0
划算+0	1.1	-0.9
划算+5	1.1	-0.9
划算+10	1.4	-0.6
划算+15	1.1	-1.0
環境好+0	1.8	-0.2
環境好+5	1.8	-0.1
環境好+10	1.9	-0.1
環境好+15	1.9	-0.1
服務好+0	1.0	-1.0
服務好+5	1.6	-0.4
服務好+10	1.6	-0.4
服務好+15	1.6	-0.3
食物精緻漂亮+0	2.0	0
食物精緻漂亮+5	2.0	0
食物精緻漂亮+10	2.0	0
食物精緻漂亮+15	2.0	0
食物很香+0	2.0	0
食物很香+5	2.0	0
食物很香+10	2.0	0
食物很香+15	2.0	0
其他正面情緒+0	1.8	-0.2
其他正面情緒+5	1.9	-0.1
其他正面情緒+10	2.0	0
其他正面情緒+15	2.0	0
份量大+0	2.0	0
份量大+5	2.0	0
份量大+10	2.0	0
份量大+15	2.0	0
小份量+0	2.0	0
小份量+5	2.0	0
小份量+10	2.0	0
小份量+15	2.0	0
重口味+0	2.0	0
重口味+5	2.0	0
重口味+10	2.0	0
重口味+15	2.0	0
口味清淡+0	2.0	0
口味清淡+5	2.1	0
口味清淡+10	2.2	0
口味清淡+15	2.1	0
偏辣+0	2.0	0
偏辣+5	2.0	0
偏辣+10	1.9	0
偏辣+15	1.9	0
偏甜+0	2.0	0
偏甜+5	2.1	0
偏甜+10	2.0	0
偏甜+15	2.0	0
偏酸+0	2.0	0
偏酸+5	2.0	0
偏酸+10	2.1	0
偏酸+15	2.2	0


# 貼餐廳評價標籤!

In [34]:
pipin=[('食物美味','食物不好吃',"非常好吃",'非常不好吃'),('划算','價格較高昂',"非常便宜",'非常貴')
       ,('環境好','環境比較不好',"環境超棒",'環境很差'),('服務好','服務較差',"服務一級棒",'服務很差')
       ,('其他正面情緒','評論有些負面情緒詞',"網路評論正面情緒非常多",'網路評論負面情緒很多')]
moods=[mood[0] for mood in pipin]
for dien in BigAnalyzeTable:
    newtag=[]
    for feature in dien["featurescores"]:
        if feature in moods:
            if dien["featurescores"][feature]>1:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[2])
            elif dien["featurescores"][feature]>0.4:
                newtag.append(feature)
            elif dien["featurescores"][feature]<-0.5:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[3])
            elif dien["featurescores"][feature]<0.0:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[1])
        else:
            if dien["featurescores"][feature]>0.7:
                newtag.append(feature)
                    
                    
#用set去除重複標籤
    dien['tags']=list(set(dien['tags']+newtag))
    print(dien['name'])
#     print(newtag)
    dien['tags'].append(dien['style'])
    print(dien['tags'])

同記安平豆花(安平總店)
['安平', '紅豆', '服務較差', '豆花', '同記安平豆花', '珍珠', '剉冰、豆花']
深藍咖啡館
['栗子', '非常貴', '派', '深藍', '食物美味', '千層', '蛋糕', '西點烘焙、麵包']
阿堂鹹粥
['虱目', '鹹粥', '魚腸', '粥', '價格較高昂', '食物美味', '阿堂', '其他小吃']
富盛號碗粿
['安平', '富盛號', '民族', '碗粿', '小份量', '食物美味', '魚羹', '其他小吃']
依蕾特布丁奶酪(安平門市)
['奶酪', '依蕾特', '抹茶', '食物美味', '典藏', '布丁', '西點烘焙、麵包']
食下有約 想法廚房
['環境好', '想法廚房', '食物精緻漂亮', '花雕', '食下有約', '區郡', '非常好吃', '食物很香', '其他正面情緒', '蹄膀', '其他異國料理']
飛饗創意西式料理
['食物精緻漂亮', '沙拉', '環境超棒', '服務好', '非常好吃', '饗', '土壤', '其他正面情緒', '燉', '典藏', '義式料理']
莉莉水果店
['莉莉', '前路', '肉圓', '豆花', '價格較高昂', '水果', '食物美味', '剉冰、豆花']
連得堂餅舖
['餅家', '煎餅', '崇安', '連得堂', '兩包', '價格較高昂', '休閒零食']
矮仔成蝦仁飯
['蝦仁', '價格較高昂', '矮仔成', '鴨蛋', '食物美味', '蛋湯', '蝦仁飯', '其他小吃']
泰成水果店
['哈密瓜', '瓜瓜', '正興', '價格較高昂', '哈蜜瓜', '食物美味', '芒果', '果汁']
騷烤家(前鋒總店)
['甜不辣', '府城騷烤家', '油條', '非常好吃', '騷烤', '其他正面情緒', '燒烤', '炭烤串燒']
阿松割包
['瘦肉', '豬舌包', '割包', '豬舌', '價格較高昂', '花生醬', '小份量', '食物美味', '麵食點心']
秉醇烘焙坊
['瓜子', '伯爵', '火山', '奶酥', '食物精緻漂亮', '食物很香', '餅乾', '其他正面情緒', '食物美味', '西點烘焙、麵包']
冰鄉
['冰', '草莓', '牛乳', '民生路', '食

神田日本料理
['綠豆湯', '份量大', '神田', '壽司', '上官', '其他正面情緒', '生魚片', '食物美味', '綜合日式料理']
漢豆府
['份量大', '豆漿', '環境好', '豆花', '剉冰', '仙草', '食物美味', '自選', '剉冰、豆花']
Malibu(二館)
['偏酸', '泰國', '阿月', '泰式料理', '環境比較不好', '食物美味', '蝦醬', '嗲', '泰式料理']
東京豚骨拉麵
['叉燒', '日本', '花月嵐', '八峰', '食物美味', '重口味', '拉麵', '日式麵食專賣']
Mr.wheat 小麥先生創意料理(新市創始店)
['創始', '小麥', '先生', '價格較高昂', '食物美味', '燉', '新市', '複合式']
泰星球
['河粉', '椰奶', '泰國菜', '泰式', '咖哩', '食物美味', '泰式料理']
金曜日咖哩專門店
['金曜日', '價格較高昂', '神農', '豬排', '日式', '咖哩', '食物美味', '其他日式料理']
陳記真正紅燒土魠魚羹
['開元', '撲克', '價格較高昂', '魚塊', '魠', '土魠魚羹', '麵食點心']
古早味蚵嗲
['蚵', '鯓', '食物美味', '鯤', '小份量', '嗲', '公車', '其他小吃']
炸蛋蔥油餅
['炸彈', '花蓮', '蔥油餅', '半熟', '花花', '小份量', '食物美味', '蔥油餅、蔥抓餅']
鈺雪冰枝店
['白河', '刨冰', '香蕉冰', '瀛', '冰店', '食物美味', '冰淇淋']
米藍花巷129異國料理餐廳
['老爺', '花巷', '西莉亞', '含附', '藍', '食物美味', '其他異國料理']
Junkies打氣吧
['四驅車', '打氣', '崑山', '果露', '價格較高昂', '氣泡', '複合式']
旺德實驗室
['飯糰', '早午餐', '食物精緻漂亮', '蚌', '偏酸', '豬排', '食物美味', '旺德', '西式早餐']
餵公子吃餅
['壽奶', '花花', '九品', '價格較高昂', '爛餅', '芝麻官', '麵食點心']
Love Ice Cream 造型馬卡龍冰淇淋
['柴犬', '可愛', '食物精緻漂

['品飲', '門市', '價格較高昂', '咖啡', '醒醒', '藍莓', '咖啡專賣']
臻皇港式餐廳
['叉燒', '環境好', '港式', '臻', '食物很香', '燒鴨', '偏甜', '食物美味', '中蝦', '粵菜 港式飲茶']
聚香園燒臘快餐
['著鴨油', '叉燒', '咩', '聚香園', '五十', '粵菜 港式飲茶']
東門町食堂
['七桃米', '鱈魚', '雞腿', '定食', '肝醋', '其他日式料理']
大丸家-手作炸雞甜甜圈
['大丸家', '圈', '甜甜', '咖哩', '炸雞', '其他日式料理']
三灣寮長盈海味屋
['魚肚', '虱目', '北門', '盈', '魚菲力', '食物美味', '海鮮餐廳']
磨磨 茶
['磨磨', '帶份', '早午餐', '冰磨', '刈包', '複合式']
文記燒臘
['香港文記燒腊', '火肉', '燒鴨', '文記', '燒臘', '其他小吃']
藍晒圖
['園區', '活動', '地點', '價格較高昂', '圖文', '重點', '酒吧 Lounge Bar']
糖菓咖啡與有時烤鬆餅
['列日', '有時', '菓', '咖啡', '鬆餅', '西式早餐']
蘇家鮮蝦餅
['名產', '蝦餅', '響鐺鐺', '訓練', '回來', '休閒零食']
豐發黑輪
['蓋飯', '價格較高昂', '謝宅', '旗魚', '黑輪', '生魚片', '食物美味', '其他小吃']
小北平餡餅粥
['北平', '小米粥', '蔥油餅', '價格較高昂', '環境比較不好', '餡餅', '佟記', '其它中式料理']
禾佳烘焙坊
['麥芽', '東海', '布蕾', '寡糖', '鳳梨', '西點烘焙、麵包']
京園壽司
['前路', '壽司', '金魚', '環境比較不好', '加州', '京園', '生魚片、壽司專賣']
黃家碗粿
['四神湯', '碗粿', '小熊', '環境比較不好', '粿', '小份量', '豬肚', '其他小吃']
3.3輕食咖啡館
['霜淇淋', '小叮噹', '價格較高昂', '文具店', '甜筒', '狡兔', '複合式']
有ㄟ輕食館
['鍋燒意麵', '李敏', '湯頭', '小高', '等湯', '其他小吃']
阿聖杏仁豆腐冰
['記者', 

拌家家酒煮義
['遠東', '吳園', '愛自', '先上去', '珠串', '咖啡專賣']
翰神溫州大餛飩
['苦苦哀求', '特餐', '大叔', '餛飩', '環境比較不好', '蒲小英', '麵食點心']
林家剉冰
['粉角', '剉冰', '湯圓', '蒲小英', '仙草', '偏甜', '剉冰、豆花']
糧戶食堂
['軟骨', '肉燥飯', '土豆', '咩', '五十', '滷肉飯']
阿秀煎粿店
['關廟', '章成麥芽餅', '大廟', '阿瑰', '粿', '小份量', '其他小吃']
和意軒(原：福洱茶棧)
['快給我', '叩', '素食', '環境比較不好', '餐聽', '萬聖節', '中式']
饗麻響辣 麻辣火鍋
['活跳跳', '響麻', '麻辣鍋', '回本', '食物美味', '肥腸', '麻辣火鍋']
紜楓烤肉刈包
['編常', '香台', '皮有', '刈包', '白皮', '刈包']
木由子茶齋
['阿奶', '老師', '夫人', '食物很香', '大禹嶺', '內心', '茶專賣']
阿良水果店
['中西區', '每日', '價格較高昂', '玉井', '水果', '芒果', '其他小吃']
蕃茄廚房
['丁香', '鍋燒意麵', '炒飯', '菜埔', '油量', '其他小吃']
香草Pasta
['手滑點', '酸辣', '香腸', '矯情', '鳥', '義式料理']
寶家味羊肉店
['羊肉', '腱子', '寶家味', '羊肉爐', '當歸', '海鮮餐廳']
泰開心泰式料理廚房
['偏辣', '鹹蛋', '香米', '拋豬', '蝦醬', '泰式', '泰式料理']
黑人洪鵝肉
['雪媽', '鵝油', '雪爸', '麵線', '包跟鵝油', '台菜餐廳']
龍鳳牛肉麵
['洪家', '牛肉麵', '價格較高昂', '龍鳳', '孫家', '南京', '麵食點心']
客再來當歸鴨
['軟骨', '當歸鴨', '紅蔗', '爌', '排骨飯', '食補藥膳餐廳']
高麗屋
['它家', '高麗館', '煎餅', '價格較高昂', '韓朝', '環境比較不好', '烤肉', '韓式料理']
呷懷念懷舊餐廳
['櫃位', '懷舊', '依夢', '價格較高昂', '時代', '館內', '台菜餐廳']
原味廚坊
['

韓巢炸雞(善化南科店)
['韓巢炸雞', '棉花糖', '善化', '價格較高昂', '雞丁', '狡兔', '韓式料理']
振香永和豆漿大王
['豆漿', '湯包', '振香', '中式早餐', '食物美味', '蛋餅', '中式早餐']
安平海鮮炒麵
['炒飯', '大火', '預防', '蛋炒飯', '公車', '熱炒、快炒']
龍龍來
['積雨雲', '筆者', '糊化', '肉粽', '刮包', '肉粽']
惠成豬心
['惠成', '豬心', '腰子', '麻油', '價格較高昂', '骨髓', '其他小吃']
No Q Pizza 美式披薩(大同店)
['博士生', '美式', '雙起司', '臘腸', '披薩', '美式料理']
冰心茶王冷飲站(台南新興店)
['冰心', '清心', '茶變', '清茶', '環境比較不好', '茶疑', '飲料專賣店']
一心王家燻羊肉
['過會', '羊肉', '稻殼', '羊皮', '燻', '其它中式料理']
飽食堂
['有點鹹', '大同路', '素肉燥', '肉絲', '榨菜', '中式']
小赤佬 干鍋店
['干鍋', '香干', '籃子', '育樂', '選辣', '環境比較不好', '食物美味', '其他鍋類']
名湘園素菜
['燴', '喜慶', '名湘園', '其他正面情緒', '壽宴', '蔬出', '中式']
阿好姨食品
['酒具', '阿好', '有良', '酒器', '姨', '休閒零食']
興豆花
['店裡長', '綿濃稠豆', '豆花', '元大', '但豆', '剉冰、豆花']
日喜美食小舖
['服務較差', '康橋', '新營', '咖嚓', '蒸蛋', '食物美味', '板橋', '其他小吃']
中壢觀光夜市泡泡冰(台南市育樂店)
['泡泡', '冰', '育樂', '現打現', '加花豆', '剉冰、豆花']
优匠 823
['澎湖', '优匠', '該店', '熬料', '土魠魚羹', '其他小吃']
歐拉廚坊
['歐拉', '食到', '地中海', '特愛他', '價格較高昂', '漫出來', '複合式']
鍋寶臭臭鍋(武聖店)
['臭臭', '鍋店', '點鍋', '鍋寶', '豆皮', '其他鍋類']
排排站專業牛排
['蝦蛄', '小編', '划算', '牛排', '

美甘素食
['', '無評', '其他小吃']
陳記牛肉湯
['', '無評', '其他小吃']
韓屋 韓式廚房
['部隊', '韓家', '韓屋韓式廚房', '韓屋', '韓式', '法式料理']
溫野菜-台南大遠百公園店
['大遠', '大麥', '五花', '野菜', '划算', '豬里', '食物美味', '涮涮鍋']
喆發涮涮鍋.火烤兩吃
['嗤', '鍋物', '喆', '肉片', '涮涮鍋', '涮涮鍋']
濰克早午餐
['份量大', '早午餐', '潛艇', '濰克', '服務好', '透亮', '食物很香', '脆薯', '划算', '食物美味', '早午餐']
尋蜜.比利
['', '無評', '西點烘焙、麵包']
佰九海產粥
['鹽蒸', '佰', '爸告', '海產', '粥', '其它中式料理']
鄭家養生飲(永大夜市店)
['', '無評', '飲料專賣店']
森菓鯛魚燒霜淇淋
['霜淇淋', '芯', '胖拔', '阿妹', '鯛魚', '冰淇淋']
竹園水果冰品簡餐專賣店
['環境好', '竹園', '麻油', '價格較高昂', '編仔', '荷包蛋', '芒果', '剉冰、豆花']
天芝然蔬食咖啡
['', '無評', '養生蔬食餐廳']
吳家羊肉 - 阿仁
['羊肉', '清寧宮', '羊肉爐', '阿仁', '吳家', '羊肉爐']
金科威補天鮮滴雞湯
['', '無評', '其他美食']
NUTS堅菓子手作工坊
['倫子', '堅果', '脆餅', '杏仁', '核桃', '甜點專賣']
ShabuSato日本涮涮鍋(台南店)
['勝博殿', '價格較高昂', '瓦城', '泰國', '豬排', '其他正面情緒', '吉米', '食物美味', '重口味', '日式']
義窩窩義式餐廳
['', '無評', '義式料理']
義起來創意廚坊
['', '無評', '義式料理']
歐派2號店
['', '無評', '甜點專賣']
泰冒煙南洋廚房
['', '無評', '泰式料理']
大豆豆嫩骨餐飲
['波妮', '清燉', '骨', '大豆', '紅燒', '麵食點心']
鮮手作麵坊
['', '無評', '麵食點心']
讚不絕口小火鍋臭臭鍋專賣
['還拿小', '趕時間', '飽天', '小火鍋', '划算', '演講', '其它

In [50]:
for i in BigAnalyzeTable:
    if i['score']>30 and i['Ncomment']>10:
        print(i['name']+' '+str(i['score']))
#         print(i['tags'])
        print(i['featurescores'])
        print('\n')
gooddien=[i for i in BigAnalyzeTable if i['score']>30 and i['Ncomment']>10]
# len(gooddien)

食下有約 想法廚房 34.61199999999995
{'食物美味': 1.36, '食物很香': 0.82, '環境好': 0.97, '食物精緻漂亮': 0.97, '份量大': 0.54, '其他正面情緒': 0.65, '划算': 0.16, '口味清淡': 0.07, '偏甜': 0.3, '服務好': 0.15, '重口味': 0.24, '小份量': 0.23, '偏辣': 0.06}


飛饗創意西式料理 35.991999999999926
{'食物美味': 1.33, '食物很香': 0.69, '划算': 0.25, '環境好': 1.06, '服務好': 0.51, '食物精緻漂亮': 1.28, '其他正面情緒': 0.79, '份量大': 0.65, '小份量': 0.1, '偏甜': 0.4, '口味清淡': 0.36, '重口味': 0.4, '偏辣': 0.12, '偏酸': 0.53}


小銅鍋義式餐廳(德安誠品店) 42.07826086956521
{'食物美味': 1.7, '食物很香': 0.99, '環境好': 1.13, '食物精緻漂亮': 0.71, '其他正面情緒': 0.59, '份量大': 0.65, '偏辣': 0.26, '重口味': 0.5, '口味清淡': 0.24, '划算': 0.09, '小份量': 0.17, '偏酸': 0.87, '服務好': 0.31, '偏甜': 0.11}


歐加里(台南大學店) 38.97999999999998
{'食物美味': 1.51, '食物很香': 0.72, '划算': 0.71, '環境好': 0.73, '食物精緻漂亮': 1.45, '其他正面情緒': 0.57, '份量大': 0.34, '小份量': 0.13, '重口味': 0.48, '偏辣': 0.12, '偏酸': 0.8, '口味清淡': 0.15, '服務好': 1.16, '偏甜': 0.3}


轉角餐廳 33.33636363636369
{'食物美味': 1.33, '食物很香': 1.0, '環境好': 1.01, '服務好': 0.6, '食物精緻漂亮': 0.82, '其他正面情緒': 0.53, '份量大': 0.33, '划算': -0.15, '重口味': 0

In [36]:
with open('D:/Data/JsonData/TainanFood/bigtable_1.2.json','w') as f:
    json.dump(BigAnalyzeTable,f)
with open('D:/Data/JsonData/TainanFood/TainanGoodDien_1.0.json','w') as f:
    json.dump(gooddien,f)   

In [37]:
# a=['食物精緻漂亮', '價格較高昂', '其他正面情緒']
# b=['食物美味', '價格較高昂', '環境好', '服務好', '食物精緻漂亮', '食物很香', '其他正面情緒', '重口味']
# c=['食物美味', '價格較高昂', '環境好', '服務好', '食物精緻漂亮', '食物很香', '其他正面情緒', '重口味']
# list(set(a+b+c))


In [18]:
import json
with open('D:/Data/JsonData/TainanFood/bigtable_1.2.json') as f:
    data=json.load(f)

In [49]:
for i in data:
    if '美味' in i['name']:
        print(i['name'])
        print(i['featurescores'])

咿吉麵包坊  隱藏版平價美味麵包店
{'食物美味': 0.71, '食物很香': 0.5, '環境好': 0.12, '食物精緻漂亮': 0.1, '其他正面情緒': 0.28, '份量大': 0.06, '划算': 0.03, '重口味': 0.1, '偏甜': 0.3, '服務好': 0.08}
美味意麵滷味
{'食物美味': 0.23, '划算': -0.16, '食物很香': 0.25}
台南 湯鼎美味鍋(永康店)
{'食物美味': 0.18}


In [23]:
data[5500]

{'Ncomment': 1,
 'TF_IDF': {'一大': 0.003353663778735919,
  '一天': 0.010127383130056566,
  '一拿結': 0.020440354628866154,
  '一隻': 0.00455315887544074,
  '一點': 0.0015124703893012483,
  '三明治': 0.03205023917373029,
  '上學': 0.010981861751180046,
  '上班': 0.004755972347336638,
  '上班族': 0.006803384398805814,
  '不小': 0.004865919520626641,
  '以外': 0.004598659968322403,
  '但別': 0.013896224288344824,
  '來給': 0.00844673573738871,
  '來說': 0.002130407771207715,
  '來頭': 0.009411094967533493,
  '個帳': 0.017847304331476684,
  '倍增': 0.01226019170321449,
  '先': 0.0012378355159383579,
  '全麥': 0.008726571816556035,
  '加入': 0.002414220106838861,
  '十分': 0.003166434294443572,
  '半糖': 0.008290743504368606,
  '原汁': 0.006089912036635601,
  '口味': 0.0023340153977166314,
  '台南市': 0.0008609931048430223,
  '吃': 0.0041159748785006495,
  '各式各樣': 0.005933197669267718,
  '同事': 0.01406818643826678,
  '吐司': 0.012876348961520663,
  '喔': 0.002189974957944981,
  '喝': 0.001160977130673277,
  '囉': 0.0015225887392619818,
  '回謹謹': 0.0